In [5]:
# Célula 1: Instalação de bibliotecas (execute esta célula primeiro)
# Instala bibliotecas Python
%pip install -q google-generativeai requests ipywidgets gTTS SpeechRecognition pydub

# Instala ffmpeg, necessário para pydub processar alguns formatos de áudio
%apt-get install -y ffmpeg

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%apt-get` not found.


In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY_1')

ModuleNotFoundError: No module named 'google'

In [ ]:
# Célula 1: Instalação de bibliotecas (execute esta célula primeiro, se ainda não o fez)
# !pip install -q google-generativeai requests ipywidgets

# Célula 2: Código do Chatbot Tutor (Corrigido)
import os
from google.colab import userdata
import google.generativeai as genai
# CORREÇÃO: Importar exceções diretamente de google.generativeai.types
from google.generativeai.types import BlockedPromptException, StopCandidateException
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import requests
import time
import threading
import re

# Configuração da API
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_1')
    genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError:
    print("Chave API 'GOOGLE_API_KEY_1' não encontrada nos Secrets do Colab.")
    GOOGLE_API_KEY = input("Por favor, insira sua Google API Key: ")
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("API Key configurada para esta sessão.")
    else:
        print("API Key não fornecida. O chatbot não poderá funcionar.")
        raise ValueError("API Key não fornecida.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")
    raise

modelo_nome = "gemini-2.0-flash"

# Estilos CSS para as mensagens
style_ia = """
    background-color: #e6f7ff; /* Azul claro para IA */
    color: #000;
    padding: 10px;
    border-radius: 10px;
    margin-top: 5px;
    margin-bottom: 5px;
    width: fit-content;
    max-width: 80%;
    word-wrap: break-word;
    align-self: flex-start;
    line-height: 1.4;
"""

style_usuario = """
    background-color: #d1ffd1; /* Verde claro para o usuário */
    color: #000;
    padding: 10px;
    border-radius: 10px;
    margin-top: 5px;
    margin-bottom: 5px;
    width: fit-content;
    max-width: 80%;
    word-wrap: break-word;
    align-self: flex-end;
    margin-left: auto;
    line-height: 1.4;
"""

# --- Criação dos Widgets ---
titulo_html_widget = widgets.HTML(
    value="<h1 style='color: #3498db; text-align: center; font-size: 38px; font-family: \"Arial Black\", Gadget, sans-serif; margin-bottom: 15px;'>Meu Tutor 🤖</h1>"
)
chat_area_vbox = widgets.VBox([])
chat_area_vbox.layout = widgets.Layout(
    max_height='500px',
    overflow_y='auto',
    padding='10px',
    border='1px solid #ddd',
    border_radius='5px',
    margin_bottom='10px'
)
input_box = widgets.Text(
    placeholder='Pergunte ao tutor...',
    layout=widgets.Layout(width='calc(100% - 110px)')
)
send_button = widgets.Button(
    description="Enviar",
    button_style='primary',
    icon='paper-plane',
    style=widgets.ButtonStyle(button_color='#3498db'),
    layout=widgets.Layout(width='100px')
)
output_area_debug = widgets.Output()

instrucao_sistema = (
    "Você é um tutor especializado em ensinar Python, Empreendedorismo, Português e Matemática para adultos, "
    "principalmente iniciantes. Sua abordagem é amigável, encorajadora e altamente adaptável. "
    "Você é um professor experiente, capaz de utilizar diferentes métodos de estudo, como a aprendizagemitiva "
    "(fazendo perguntas que estimulem o raciocínio, propondo pequenos desafios práticos) e passiva "
    "(fornecendo explicações claras, analogias e exemplos concretos). "
    "Você consegue adaptar sua linguagem, o nível de complexidade e os exemplos para se adequar às necessidades e ao "
    "estilo de aprendizado de cada aluno, buscando sempre a forma mais eficaz e compreensível de ensinar."
    "\n\n"
    "Seu fluxo de interação principal é:"
    "1. Apresente-se brevemente e pergunte ao aluno qual dos temas (Python, Empreendedorismo, Português, Matemática ou inglês) "
    "ele gostaria de explorar primeiro. **Esta etapa já será simulada pela interface com uma mensagem inicial, então sua primeira resposta real deve ser reagindo à escolha do tema pelo aluno.**"
    "2. Após o aluno escolher o tema, procure entender o nível de conhecimento dele sobre o assunto. Você pode fazer perguntas como: "
    "\'Você já teve algum contato com [tema] antes?\', \'Há algo específico em [tema] que você já tentou aprender ou tem curiosidade?\' "
    "ou \'Como você se sente em relação à sua familiaridade com [tema]: totalmente iniciante, já vi alguma coisa, ou já tenho alguma base?\'. "
    "Analise as respostas para inferir o nível."
    "3. Com base nesse nível, proponha uma \'trilha de conhecimento\' personalizada. Esta trilha deve ser um pequeno roteiro de tópicos "
    "ou módulos a serem estudados em uma sequência lógica para aquele nível."
    "4. Ao apresentar a trilha, deixe MUITO CLARO que é apenas uma SUGESTÃO. Pergunte explicitamente ao aluno: "
    "\'Esta é uma sugestão de trilha que podemos seguir. O que você acha? Gostaria de começar pelo primeiro tópico, "
    "pular para algum outro, ou talvez focar em algo diferente que você tinha em mente dentro de [tema]?\'"
    "5. Ao longo das explicações, após cada conceito chave, incentive o aluno a compartilhar suas interpretações, "
    "dúvidas ou exemplos. Forneça feedback construtivo, corrija equívocos com paciência e responda perguntas "
    "de forma clara, usando mais exemplos se necessário."
    "6. Mantenha um tom conversacional e encorajador. Use emojis de forma apropriada para deixar a conversa mais leve, se achar pertinente."
    "7. Seu objetivo é criar uma experiência de aprendizado que se assemelhe a uma tutoria individual, "
    "permitindo que o aluno explore o conteúdo no seu próprio ritmo e faça perguntas livremente."
    "\n\n"
    "**Todas as suas respostas e interações devem ser estritamente em português do Brasil.**"
)

model = genai.GenerativeModel(
    model_name=modelo_nome,
    system_instruction=instrucao_sistema
)
chat = None
local_history_display = []

def exibir_mensagem_no_chat(autor, mensagem_texto):
    mensagem_html = mensagem_texto.replace('\n', '<br>')
    mensagem_html = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', mensagem_html)
    mensagem_html = re.sub(r'\*(.*?)\*', r'<i>\1</i>', mensagem_html)
    mensagem_html = re.sub(r'```(.*?)```', r'<pre style="background-color: #f0f0f0; color: #333; padding: 10px; border-radius: 5px; white-space: pre-wrap; word-wrap: break-word;">\1</pre>', mensagem_html, flags=re.DOTALL)
    mensagem_html = re.sub(r'`([^`]+)`', r'<code style="background-color: #f0f0f0; color: #c7254e; padding: 2px 4px; border-radius: 3px; font-family: monospace;">\1</code>', mensagem_html)
    style = style_usuario if autor == "Você" else style_ia
    msg_widget = widgets.HTML(f'<div style="{style}">{mensagem_html}</div>')
    chat_area_vbox.children = tuple(list(chat_area_vbox.children) + [msg_widget])

robot_char = "🤖"
point_char = "•"
parar_animacao = False
thinking_widget = widgets.HTML(value="")
animacao_thread = None

def gerar_frame_pensando(frame_number):
    num_points = 3
    active_point = frame_number % num_points
    points_str = "".join([f"<span style='opacity: {1 if i == active_point else 0.3};'>{point_char}</span>" for i in range(num_points)])
    frame = f"<div style='font-size: 16px; color: #555; text-align: left; margin-left: 10px; margin-top: 5px; margin-bottom: 5px;'>{robot_char} digitando {points_str}</div>"
    return frame

def animar(widget_para_animar):
    global parar_animacao
    frame = 0
    while not parar_animacao:
        if not widget_para_animar in chat_area_vbox.children:
            break
        try:
            widget_para_animar.value = gerar_frame_pensando(frame)
        except Exception:
            break
        time.sleep(0.3)
        frame += 1
    if widget_para_animar in chat_area_vbox.children:
        try:
            widget_para_animar.value = ""
            chat_area_vbox.children = tuple(c for c in chat_area_vbox.children if c != widget_para_animar)
        except Exception:
            pass

def iniciar_animacao_pensando():
    global parar_animacao, animacao_thread, thinking_widget
    parar_animacao = False
    thinking_widget.value = ""
    if thinking_widget not in chat_area_vbox.children:
        chat_area_vbox.children = tuple(list(chat_area_vbox.children) + [thinking_widget])
    animacao_thread = threading.Thread(target=animar, args=(thinking_widget,))
    animacao_thread.start()

def parar_animacao_pensando():
    global parar_animacao, animacao_thread
    parar_animacao = True
    if animacao_thread and animacao_thread.is_alive():
        animacao_thread.join(timeout=0.5)
    if thinking_widget in chat_area_vbox.children:
        try:
            chat_area_vbox.children = tuple(child for child in chat_area_vbox.children if child != thinking_widget)
        except Exception:
             pass

def enviar_mensagem_chat_callback(button_or_text_event):
    mensagem_usuario = input_box.value
    if not mensagem_usuario.strip():
        return
    input_box.value = ''
    exibir_mensagem_no_chat("Você", mensagem_usuario)
    local_history_display.append({"role": "user", "parts": [mensagem_usuario]})
    iniciar_animacao_pensando()
    try:
        if not chat:
            print("Erro: Chat não inicializado.")
            parar_animacao_pensando()
            exibir_mensagem_no_chat("IA", "Desculpe, ocorreu um erro interno (chat não iniciado).")
            return

        resposta_ia = chat.send_message(mensagem_usuario)
        texto_resposta_ia = resposta_ia.text
        local_history_display.append({"role": "model", "parts": [texto_resposta_ia]})
        parar_animacao_pensando()
        exibir_mensagem_no_chat("IA", texto_resposta_ia)

    # CORREÇÃO: Usar as exceções importadas diretamente
    except BlockedPromptException as e:
        texto_resposta_ia = "Sua mensagem foi bloqueada. Por favor, reformule sua pergunta de forma neutra."
        local_history_display.append({"role": "model", "parts": [texto_resposta_ia]})
        parar_animacao_pensando()
        exibir_mensagem_no_chat("IA", texto_resposta_ia)
        with output_area_debug:
            output_area_debug.clear_output()
            print(f"BlockedPromptException: {e}")
    except StopCandidateException as e: # Anteriormente types.generation_types.StopCandidateException
        texto_resposta_ia = "A resposta foi interrompida ou considerada insegura. Tente perguntar de forma mais concisa ou diferente."
        local_history_display.append({"role": "model", "parts": [texto_resposta_ia]})
        parar_animacao_pensando()
        exibir_mensagem_no_chat("IA", texto_resposta_ia)
        with output_area_debug:
            output_area_debug.clear_output()
            print(f"StopCandidateException: {e}")
    except Exception as e:
        texto_resposta_ia = f"Desculpe, ocorreu um erro ao processar sua mensagem. (Detalhe: {type(e).__name__})"
        local_history_display.append({"role": "model", "parts": [texto_resposta_ia]})
        parar_animacao_pensando()
        exibir_mensagem_no_chat("IA", texto_resposta_ia)
        with output_area_debug:
            output_area_debug.clear_output()
            print(f"Ocorreu um erro inesperado: {e}")
            import traceback
            traceback.print_exc()

send_button.on_click(enviar_mensagem_chat_callback)
input_box.on_submit(enviar_mensagem_chat_callback)

def iniciar_conversa_com_ia():
    global chat
    chat_area_vbox.children = ()
    local_history_display.clear()
    chat = model.start_chat(history=[])
    initial_ia_message = (
        "Olá! 👋 Sou o seu Tutor Robô, pronto para te ajudar a aprender sobre Python, Empreendedorismo, Português, Matemática ou Inglês. "
        "Para começarmos, qual desses temas você gostaria de explorar primeiro?"
    )
    local_history_display.append({"role": "model", "parts": [initial_ia_message]})
    exibir_mensagem_no_chat("IA", initial_ia_message)

input_hbox = widgets.HBox([input_box, send_button])
descricao_html_widget = widgets.HTML(
    value="<p style='text-align: center; color: #555; font-size: 14px;'>Digite sua dúvida ou escolha um tema para começar seu aprendizado!</p><br>"
)
main_ui = widgets.VBox([
    titulo_html_widget,
    descricao_html_widget,
    chat_area_vbox,
    input_hbox,
    # output_area_debug
], layout=widgets.Layout(width='70%', margin='20px auto', padding='20px', border='1px solid #ddd', box_shadow='0 4px 8px 0 rgba(0,0,0,0.1)'))

display(main_ui)
iniciar_conversa_com_ia()

ModuleNotFoundError: No module named 'google'